In [1]:
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import chainer
from skimage.measure import compare_ssim as ssim
import chainer.functions as F
from net import SeqResVAE, SeqVAE
%matplotlib inline

In [2]:
FILE_PATH = Path('.').resolve().parent
ROOT_PATH = FILE_PATH.parent
RESULT_PATH = ROOT_PATH.joinpath('results/SeqVAE')
MODEL_PATH = ROOT_PATH.joinpath('models/SeqVAE')
print(FILE_PATH)
print(ROOT_PATH)
print(RESULT_PATH)
print(MODEL_PATH)

/home/shimo/CMULAB/Spatio-Temporal-SR/experiments
/home/shimo/CMULAB/Spatio-Temporal-SR
/home/shimo/CMULAB/Spatio-Temporal-SR/results/SeqVAE
/home/shimo/CMULAB/Spatio-Temporal-SR/models/SeqVAE


In [3]:
# obj = 1

# print(data_path)
all_obj_images = []

for obj in range(1, 100 +1):
    data_path = ROOT_PATH.joinpath('datasets', 'seq-coil-100', 'obj{}'.format(obj))
    img_path = data_path.joinpath('obj{}__0.png'.format(obj))
    img = np.array(Image.open(img_path), dtype=np.float32).transpose(2, 0, 1) / 255.0
    all_obj_images.append(img)

all_obj_images = np.array(all_obj_images)
print(all_obj_images.shape, all_obj_images.dtype)

(100, 3, 128, 128) float32


In [4]:
seq14_images = []
for i in range(72):
    data_path = ROOT_PATH.joinpath('datasets', 'seq-coil-100', 'obj14')
    img_path = data_path.joinpath('obj14__{}.png'.format(i * 5))
    img = np.array(Image.open(img_path), dtype=np.float32).transpose(2, 0, 1) / 255.0
    seq14_images.append(img)
    
seq14_images = np.array(seq14_images)
print(seq14_images.shape, seq14_images.dtype)

(72, 3, 128, 128) float32


In [5]:
vae = SeqVAE(128, 100, 16)
seqvae = SeqVAE(128, 100, 16)
chainer.serializers.load_npz(
    '/home/shimo/CMULAB/Spatio-Temporal-SR/models/SeqVAE/preSeqVAE_latent100_coef1.0_ch16.npz', vae)
chainer.serializers.load_npz(
    '/home/shimo/CMULAB/Spatio-Temporal-SR/models/SeqVAE/SeqVAE_latent100_coef11.0_coef11.0_ch16.npz', seqvae)

In [6]:
# 再構成 誤差

In [7]:
x_gt = all_obj_images \
        .reshape(10, 10, 3, 128, 128).transpose(0, 3, 1, 4, 2).reshape(10 * 128, 10 * 128, 3)
x_gt = np.clip(x_gt * 255, 0, 255).astype(np.uint8)
Image.fromarray(x_gt).save('Gt_rec.png')

with chainer.using_config('train', False), chainer.no_backprop_mode():
    x_rec = seqvae(all_obj_images).array \
            .reshape(10, 10, 3, 128, 128).transpose(0, 3, 1, 4, 2).reshape(10 * 128, 10 * 128, 3)
    x_rec = np.clip(x_rec * 255, 0, 255).astype(np.uint8)
    Image.fromarray(x_rec).save('seqVAE_rec.png')

In [8]:
with chainer.using_config('train', False), chainer.no_backprop_mode():
    x_rec = vae(all_obj_images).array \
            .reshape(10, 10, 3, 128, 128).transpose(0, 3, 1, 4, 2).reshape(10 * 128, 10 * 128, 3)
    x_rec = np.clip(x_rec * 255, 0, 255).astype(np.uint8)
    Image.fromarray(x_rec).save('VAE_rec.png')

In [17]:
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr

In [20]:
x_gt = all_obj_images \
        .reshape(100, 3, 128, 128).transpose(0, 2, 3, 1)
x_gt = np.clip(x_gt * 255, 0, 255).astype(np.uint8)

with chainer.using_config('train', False), chainer.no_backprop_mode():
    x_seq = seqvae(all_obj_images).array \
            .reshape(100, 3, 128, 128).transpose(0, 2, 3, 1)
    x_seq = np.clip(x_seq * 255, 0, 255).astype(np.uint8)

    
with chainer.using_config('train', False), chainer.no_backprop_mode():
    x_rec = vae(all_obj_images).array \
            .reshape(100, 3, 128, 128).transpose(0, 2, 3, 1)
    x_rec = np.clip(x_rec * 255, 0, 255).astype(np.uint8)

In [34]:
vae_psnrs = []
vae_ssims = []
seq_psnrs = []
seq_ssims = []

for i in range(100):
    vae_ssims.append(compare_ssim(x_gt[i], x_rec[i], multichannel=True))
    vae_psnrs.append(compare_psnr(x_gt[i], x_rec[i]))
    seq_ssims.append(compare_ssim(x_gt[i], x_seq[i], multichannel=True))
    seq_psnrs.append(compare_psnr(x_gt[i], x_seq[i]))

print('VAE PSNR mean:', np.array(vae_psnrs).mean())
print('VAE SSIM mean:', np.array(vae_ssims).mean())

print('SeqVAE PSNR mean:', np.array(seq_psnrs).mean())
print('SeqVAE SSIM mean:', np.array(seq_ssims).mean())

VAE PSNR mean: 28.248479136767838
VAE SSIM mean: 0.8509177635796477
SeqVAE PSNR mean: 26.700266431272997
SeqVAE SSIM mean: 0.8140420913109006


In [9]:
in_num = 0
out_num = 10 * 5

seq_in_images = []
seq_out_images = []

for obj in [1, 6, 12, 14, 37, 48, 52, 65, 74, 100]:
    data_path = ROOT_PATH.joinpath('datasets', 'seq-coil-100', 'obj{}'.format(obj))
    img_in_path = data_path.joinpath('obj{}__{}.png'.format(obj, in_num))
    img = np.array(Image.open(img_in_path), dtype=np.float32).transpose(2, 0, 1) / 255.0
    seq_in_images.append(img)
    
    img_out_path = data_path.joinpath('obj{}__{}.png'.format(obj, out_num))
    img = np.array(Image.open(img_out_path), dtype=np.float32).transpose(2, 0, 1) / 255.0
    seq_out_images.append(img)

seq_out_images = np.array(seq_out_images)
print(seq_out_images.shape, seq_out_images.dtype)
seq_in_images = np.array(seq_in_images)
print(seq_in_images.shape, seq_in_images.dtype)

(10, 3, 128, 128) float32
(10, 3, 128, 128) float32


In [10]:
with chainer.using_config('train', False), chainer.no_backprop_mode():
    z_in = seqvae.encode(seq_in_images)[0].array
    z_out = seqvae.encode(seq_out_images)[0].array
    z_seq = z_in + np.kron(np.linspace(0, 1, 10).astype(np.float32).reshape(10, 1, 1), (z_out - z_in))
    x_seq = seqvae.decode(z_seq.reshape(100, 100)).array\
            .reshape(10, 10, 3, 128, 128).transpose(1, 3, 0, 4, 2).reshape(10 * 128, 10 * 128, 3)
    
x_seq = np.clip(x_seq * 255, 0, 255).astype(np.uint8)
Image.fromarray(x_seq).save('seqVAE_seq.png')

In [11]:
with chainer.using_config('train', False), chainer.no_backprop_mode():
    z_in = vae.encode(seq_in_images)[0].array
    z_out = vae.encode(seq_out_images)[0].array
    z_seq = z_in + np.kron(np.linspace(0, 1, 10).astype(np.float32).reshape(10, 1, 1), (z_out - z_in))
    x_seq = vae.decode(z_seq.reshape(100, 100)).array\
            .reshape(10, 10, 3, 128, 128).transpose(1, 3, 0, 4, 2).reshape(10 * 128, 10 * 128, 3)
    
x_seq = np.clip(x_seq * 255, 0, 255).astype(np.uint8)
Image.fromarray(x_seq).save('VAE_seq.png')